In [1]:
import torch
import os
import numpy as np
import pandas as pd
from warnings import simplefilter
from torch.utils.data import DataLoader

simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=FutureWarning)

c:\Users\20192584\Anaconda3\envs\CNN2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

In [3]:
from DataLoader import *
from Trainer import *
from Model import *
from RESNET import *

In [4]:
dtype = torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Split the data into train, validation and test set
dataspliter = DataSpliter(DATA_DIR, device, train_frac=0.6, val_frac=0.2, test_frac=0.2, use_seed=True, seed_val=42)

# Load Data from DataLoader
train_set = dataspliter.train
val_set = dataspliter.val
test_set = dataspliter.test

train_dataset = CustomImageDataset(DATA_DIR, train_set)
val_dataset = CustomImageDataset(DATA_DIR, val_set)
test_dataset = CustomImageDataset(DATA_DIR, test_set)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [7]:
# Get the model
model = CNNModel()
trainer = Trainer(model, train_dataloader, val_dataloader, test_dataloader)

The device that will be used in training is Quadro P1000


In [8]:
trainer.fit(epochs=10, batch_size= 8)
trainer.save_model("model.pt", "savefolderpytorch")

Epoch 1


Training:   8%|▊         | 24/288 [00:11<02:11,  2.01it/s, loss=43.1]   


KeyboardInterrupt: 

In [ ]:
_, output, labels = trainer.val_epoch(test_dataloader)

Validation: 768it [01:13, 10.39it/s, loss=3.76]                       

Validation loss is 0.056640660700698696


In [ ]:
dfresults = pd.DataFrame(np.append(output, labels, axis=1),columns=["sigmapred", "sigmatrue"])
dfresults["RMSE"] = np.sqrt((dfresults["sigmapred"]-dfresults["sigmatrue"])**2)
dfresults

,sigmapred,sigmatrue,RMSE
0,70.291328,69.00,1.291328
1,70.291328,72.25,1.958672
2,70.291328,74.00,3.708672
3,70.291328,70.00,0.291328
4,70.291328,68.75,1.541328
...,...,...,...
763,70.291328,67.25,3.041328
764,70.291328,69.00,1.291328
765,70.291328,70.00,0.291328
766,70.291328,71.00,0.708672
